Adding numerical data to the dataset:
* Metascore (rating giving only by movie critics)
* Number of votes for the rating

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import imdb
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup

In [ ]:
# Dataset with test dataset shortened to the first 20 movies
dataset = pd.read_csv('new_dataset.csv')
test_df = dataset.iloc[:20, :]

In [ ]:
def searching_IMDb_movie(title_column, dataset):
    """
    Updating the dataset by adding two new columns for each movie 
    including metascore and number of ratings. 
    
    Parameters:
    name_column : the column containing the person's name
    dataset : dataset
    person_type : must specify for IMDb if person is 'actor'/'director'
    """
    # 1. Instantiating imdb 
    ia = imdb.IMDb()

    # 2. For each row of the df, extract the nam for the specific column
    for index, row in dataset.iterrows():
        try:
            # Get the movie title
            movie_title = row[title_column]
            
            # Search for a movie on IMDb by title
            movie = ia.search_movie(movie_title)
            movie_id = movie[0].getID()            

            # Make a request to the URL and get the HTML content

            # Request for movie (different webpage)
            movie_req = Request(
                url = f'https://www.imdb.com/title/tt{movie_id}/', 
                headers={'User-Agent': 'Mozilla/5.0'}
            )
            movie_html = urlopen(movie_req).read()
            movie_soup = BeautifulSoup(movie_html, 'html.parser')

            # Request for metascore (different webpage)
            meta_req = Request(
                url = f'https://www.imdb.com/title/tt{movie_id}/criticreviews/?ref_=tt_ov_rt', 
                headers={'User-Agent': 'Mozilla/5.0'}
            )
            meta_html = urlopen(meta_req).read()
            meta_soup = BeautifulSoup(meta_html, 'html.parser')
            
            # If metascore exists try to extract info
            try:
                metascore_line = meta_soup.find('div', {'class': 'sc-79ae5a4-0 kUTYKi'}).text.split()
                metascore = get_value(metascore_line[0].split('M')[0])
            except AttributeError:
                metascore = None

            # If number of votes exists try to extract info
            try:
                num_votes_line = movie_soup.find('div', {'class': 'sc-e457ee34-3 frEfSL'}).text.split()
                num_votes = get_value(num_votes_line[0])
            except AttributeError:
                num_votes = None

        except imdb._exceptions.IMDbError:
            metascore = None
            num_votes = None

        # Make a new column and append the new data
        dataset.loc[index, f'{title_column}_metascore'] = metascore
        dataset.loc[index, f'{title_column}_num_votes'] = num_votes
    
        print(index)

In [ ]:
searching_IMDb_movie('Title', test_df)
